# Cooperative Interaction on JudgeBench with Position Swapping


Please see SOURCES.txt for further source information.

## Imports

In [ ]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, GroupChat, GroupChatManager
import re
from tqdm import tqdm

## Data

In [ ]:
# Load and prepare a subset of the JudgeBench dataset
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")
df = pd.DataFrame(JudgeBench_Claude)
df_sampled = df.sample(n=100, random_state=42).reset_index(drop=True)
df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

## Config

In [ ]:
# Load Azure OpenAI configuration from environment variables
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION")

# Start of code citation [1]
# The following code is adapted from the source above.

# Define the model configuration for Azure OpenAI API access
config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,
        "temperature": 0,
        "cache_seed": 42
    }
]

llm_config={"config_list": config_list}

# End of code citation [1]

## System Design

In [ ]:
# Define the system prompt for the Analyst-Agent
agent_1_system_message =f"""
You are an Analyst-Agent in a cooperative reasoning chain.
Your task is to make an initial assessment whether Response A or Response B is more factually correct.
Give an explanation on your initial assessment using about 200 words.
Always begin your output with: "As an Analyst-Agent in an initial assessment, I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

# Define the system prompt for the Critical-Enhancer-Agent
agent_2_system_message =f"""
You are a Critical-Enhancer-Agent in a cooperative reasoning chain.
Your task is to carefully review the Analyst-Agent's initial assessment and improve upon it.
You may add missing facts, point out inaccuracies or refine the reasoning in a constructive way.
Give an explanation for your review and your suggestions for improvement using about 200 words.
Always begin your output with: "Building on the Analyst-Agent's initial assessment, I ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

# Define the system prompt for the Decision-Agent
agent_3_system_message =f"""
You are the final Decision-Agent in a cooperative reasoning chain.
Your task is to integrate the insights from the Analyst-Agent and the Critical-Enhancer-Agent into one final judgement.
Give an explanation for your final judgement using about 200 words.
Always begin your output with: "As a final Decision-Agent, after carefully reviewing the Analyst-Agents' and Critical-Enhancer-Agents' previous analyses, I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

In [ ]:
# Start of code citation [1]
# The following code is adapted from the source above.

# Initialize the ConversableAgents for system setup
agent_1 = ConversableAgent(
    name="Analyst-Agent",
    llm_config=llm_config,
    system_message=agent_1_system_message,
    human_input_mode="NEVER",
)

agent_2 = ConversableAgent(
    name="Critical-Enhancer-Agent",
    llm_config=llm_config,
    system_message=agent_2_system_message,
    human_input_mode="NEVER",
)

agent_3 = ConversableAgent(
    name="Decision-Agent",
    llm_config=llm_config,
    system_message=agent_3_system_message,
    human_input_mode="NEVER",
)

# Initialize the GroupChat and GroupChatManager for system setup
group_chat = GroupChat(
    agents=[agent_1, agent_2, agent_3],
    messages=[],
    max_round=4,
    speaker_selection_method="round_robin"
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

# End of code citation [1]

## Evaluation

In [ ]:
# Define the evaluation function
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    # Start of code citation [1]
    # The following code is adapted from the source above.

    chat_results = agent_3.initiate_chat(
        group_chat_manager,
        message=message,
        summary_method="last_msg",
    )

    # End of code citation [1]

    result_str = str(chat_results.chat_history[-1]["content"])

    pattern = r'"response"\s*:\s*"?([AB])"?'

    match = re.search(pattern, result_str)
    # If pattern not found, assign "X" to indicate invalid response
    system_decision = match.group(1) if match else "X"

    ground_truth = "A" if "A>" in label else "B"

    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }

In [ ]:
# Prepare evaluation data
num_rows = 100
df_final_subset = df_final.head(num_rows)

# Evaluate responses in normal position
results_1 = []
for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/cooperative_1.csv', index=False)

# Evaluate responses in swapped positions
results_2 = []
for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"],  # swap positions
        response_B=row["response_A"],  # swap positions
        label=row["label"],
    )

    # Correct system decision and correctness flag after swapping
    # If system_decision is "X", leave it unchanged
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/cooperative_2.csv', index=False)